In [ ]:
!pip install kafka-python

In [ ]:
import os
import getpass
import sys
import json
import base64
import kafka
import ssl
import time
import matplotlib.pyplot as plt
import io
from PIL import Image

EVENTSTREAMS_TOPIC = 'DefaultTopic'
SHOW_IMAGES = False

In [ ]:
creds_string = getpass.getpass()
creds = json.loads(creds_string)


In [ ]:
# Connect to EventStreams, with our loaded credentials and attach to the requested Topic.
cons = None
while cons is None:
    try:
        cons = kafka.KafkaConsumer(EVENTSTREAMS_TOPIC, \
                                   bootstrap_servers=creds["kafka_brokers_sasl"], \
                                   security_protocol="SASL_SSL", \
                                   sasl_mechanism="PLAIN", \
                                   sasl_plain_username=creds["user"], \
                                   sasl_plain_password=creds["api_key"], \
                                   ssl_cafile=ssl.get_default_verify_paths().cafile, \
                                   auto_offset_reset='latest')
        print("Connected to Broker.")
    except kafka.errors.NoBrokersAvailable:
        print("No Brokers Available. Retrying ...")
        time.sleep(1)
        cons = None


In [ ]:
%matplotlib inline

did = 0
while True:
    try:
        parts = cons.poll(10000, max_records=15)
        for tp in parts:
            for item in parts[tp]:
                m = json.loads(item.value.decode('utf-8'))
                
                if 'image' in m and SHOW_IMAGES:
                    oimg = base64.b64decode(m['image'])
                    pimg = m['prepared_image']
                    f = io.BytesIO(oimg)
                    oi = Image.open(f)
                    print(m['timestamp'], m['camera'],m['result_class'],m['result_probability'],m['prep_time'],m['predict_time'])
                    for i,p in enumerate(m['predictions']):
                        print("  %2d: %.6f" %(i,p))
                    plt.close()
                    plt.subplot(1,2,1)
                    plt.imshow(pimg, cmap=plt.cm.gray_r)#, interpolation='lanczos')
                    plt.title('Prepared Image, Predicted ' + str(m['result_class']))
                    plt.subplot(1,2,2)
                    plt.imshow(oi)
                    plt.title("Original Image")
                    plt.show()
                    oi.close()
                    print("\n")
                    did += 1
                elif 'camera_metrics' in m:
                    print("Interval:", m['timestamp'])
                    print("Got Classification Metrics for the last interval:")
                    total = 0
                    for k in m['camera_metrics']:
                        print("    ",k)
                        print("        Certain counts:   ", m['camera_metrics'][k]['certain'])
                        print("        Uncertain counts: ", m['camera_metrics'][k]['uncertain'])
                        total += sum(m['camera_metrics'][k]['certain']) + sum(m['camera_metrics'][k]['uncertain'])
                    print("Classified images in last interval: ", total)
                    print("    Prep Latencies:")
                    for k in m['latency_metrics']['prep']:
                        print("        %-10s: %s"%(k, m['latency_metrics']['prep'][k]) )
                    print("    Prediction Latencies:")
                    for k in m['latency_metrics']['predict']:
                        print("        %-10s: %s"%(k, m['latency_metrics']['predict'][k]) )
                    print("\n")
                    
                    if 'config' in m:
                        dur = m['config']['metrics_duration']
                        td = m['config']['delay']
                        par = m['config']['classify_parallel']
                        print(m['config'])

                        # Prep rate (total rate)
                        Rp = total / dur
                                        
                        # Score rate (per parallel path)
                        Rs = total / dur / par
                    
                        # Ingest overhead average time
                        ti = 1/Rp - td - m['latency_metrics']['prep']['mean']
                    
                        # Queing/parallelism/cross-PE overhead average time
                        tq = 1/Rs - m['latency_metrics']['predict']['mean']
                    
                        print("Average Image Rate (overall):  ", Rp)
                        print("Average Score Rate (per path): ", Rs)
                        print("Mean Ingest overhead time:     ", ti)
                        print("Mean Parallelism overhead time:", tq)
                        print("\n")
                    
                    
                    did += 1
    except Exception as e:
        print(e)
